In [ ]:
import time
import torch
import numpy as np
import networkx as nx
import collections
import matplotlib.pyplot as plt
from data.gene_graphs import GeneManiaGraph
from scipy import sparse
import joblib 
import sklearn.cluster
import scipy.cluster
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import ward, fcluster
from sklearn.cluster import AgglomerativeClustering


In [5]:
adj = torch.FloatTensor([[1, 1, 0], [1, 1, 0], [0, 0, 1]])
print(adj)

tensor([[1., 1., 0.],
        [1., 1., 0.],
        [0., 0., 1.]])


In [6]:
x = torch.FloatTensor([[2, 3, 4], [5, 6, 7]])
print(x)
print(x.shape)

tensor([[2., 3., 4.],
        [5., 6., 7.]])
torch.Size([2, 3])


In [7]:
print(x.size(-1))
new_x = x[0].view(1, x.size(-1))
print(new_x)

3
tensor([[2., 3., 4.]])


In [8]:
res =  new_x * adj
print(res)

tensor([[2., 3., 0.],
        [2., 3., 0.],
        [0., 0., 4.]])


In [9]:
res.max(dim=1)[0]

tensor([3., 3., 4.])

In [17]:
adj = torch.FloatTensor([[1, 1, 0], [1, 1, 0], [0, 0, 1]])
print(adj)

tensor([[1., 1., 0.],
        [1., 1., 0.],
        [0., 0., 1.]])


In [18]:
x = torch.FloatTensor([[2, 3, 4], [5, 6, 7]])
print(x)
print(x.shape)

tensor([[2., 3., 4.],
        [5., 6., 7.]])
torch.Size([2, 3])


In [19]:
print(x.size(-1))
new_x = x[0].view(1, x.size(-1))
print(new_x)

3
tensor([[2., 3., 4.]])


In [20]:
res =  new_x * adj
print(res)

tensor([[2., 3., 0.],
        [2., 3., 0.],
        [0., 0., 4.]])


In [21]:
res.max(dim=1)[0]

tensor([3., 3., 4.])

In [17]:
gene_graph = GeneManiaGraph()
neighbors = gene_graph.bfs_sample_neighbors("RPL13", 100)
adj = np.asarray(nx.to_numpy_matrix(neighbors))

Torrent name: genemania.pkl, Size: 9.61MB


In [18]:
sparseadj = sparse.csr_matrix(adj)


In [19]:
np.fill_diagonal(adj, 1.)  # TODO: Hummm, think it's a 0.
D = adj.sum(axis=1)


In [20]:
D_inv = np.diag(1. / np.sqrt(D))


In [21]:
D_inv_sparse = sparse.csr_matrix(D_inv)
adj_sparse = sparse.csr_matrix(adj)

In [22]:
norm_transform = D_inv_sparse.dot(adj_sparse).dot(D_inv_sparse)


In [23]:
norm_transform

<100x100 sparse matrix of type '<class 'numpy.float64'>'
	with 1294 stored elements in Compressed Sparse Row format>

In [ ]:
x = torch.FloatTensor(np.random.rand(10, 640, 16000))
print(x.shape)

print(x[0])
print(x[0].shape)


In [ ]:
print(x.size(-1))
new_x = x[0]
print(new_x)
print(new_x.shape)

In [ ]:
temp = []
start = time.time()
new_x = new_x.cuda(1)
for i in range(640):
    temp.append((new_x[i].view(1, 16000, 1) * adj).max(dim=1)[0].cpu())
print(time.time() - start)
res = torch.stack(temp).cuda()

In [32]:
res.shape

torch.Size([32, 1, 16000])

In [ ]:
(new_x[0].view(1, 16000, 1).cuda() * adj.cuda()).shape

In [ ]:
%timeit -n 1 -r 2 new_x[0].view(1, 16000, 1).cuda() * adj.cuda()

In [ ]:

            # import pdb; pdb.set_trace()
            # start = time.time()
            # max_value = (x.view(-1, x.size(-1), 1) * adj).max(dim=1)[0]



In [ ]:
        # import pdb; pdb.set_trace()
        # start = time.time()
        #
        # for i in range(x.shape[0]):
        #     temp[i] = (x[i] * adj).max(dim=1)[0]
        # print(str(time.time() - start))
        # max_value = temp
        # import pdb; pdb.set_trace()


In [ ]:
            # temp = []
            # x_cpu = x.view(-1, x.size(-1)).cpu()
            # start = time.time()
            # print("starting")
            # import pdb; pdb.set_trace()
            # 
            # for row in adj.t():
            #     neighbors = row.nonzero().view(-1)
            #     temp.append(x_cpu[:, neighbors].max(dim=1)[0].cuda())
            # print(time.time() - start)
            # max_value = torch.stack(temp).t()
            # 
            # what we used to do: it runs out of memeory
            # max_value = (x.view(-1, x.size(-1), 1) * adj.cuda()).max(dim=1)[0]


In [27]:
temp = adj[:5000, :5000]

In [29]:
sparse_temp = sparse.csr_matrix(temp)

In [30]:
%timeit sklearn.cluster.AgglomerativeClustering(n_clusters=2000, affinity='euclidean', memory='/tmp', connectivity=(sparse_temp > 0.).astype(bool), compute_full_tree='auto', linkage='ward').fit_predict(temp.astype("float32"))


281 ms ± 2.24 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
